<h1><center> Json Data Processing with MongoDB and Python</center></h1>
<h3><center> By: Gyan Prakash Tripathi</center></h3>
# Step 0: Installing and importing required packages

In [1]:
!pip install pymongo
import pymongo

    100% |████████████████████████████████| 419kB 763kB/s ta 0:00:01
You are using pip version 9.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Step 1: Creating MongoClient instance and creating new database

In [2]:

myclient = pymongo.MongoClient("mongodb://localhost:27017/")

mydb = myclient["mydatabase"]

In [3]:
print(myclient.list_database_names())

['admin', 'config', 'local']


Here we can see only three databases, it is so because we have no value in our new database. So, it is not displayed.

In [6]:
# creating a new collection (for testing)

mycol = mydb["companies"]

In [7]:
print(mydb.list_collection_names())

[]


Again, we dont have any data in our collection so it is blank. Everything works fine for now. Let's proceed.
# Step 2: Inserting json data into the database

In [98]:
'''
#Real Task
import json
mycol=mydb['companies']
with open('companies.json') as f:
    file_data = json.load(f)
    # use collection_currency.insert(file_data) if pymongo version < 3.0
    mycol.insert(file_data)  
'''

"\n#Real Task\nimport json\nmycol=mydb['companies']\nwith open('companies.json') as f:\n    file_data = json.load(f)\n    # use collection_currency.insert(file_data) if pymongo version < 3.0\n    mycol.insert(file_data)  \n"

*We could use above approach for insertion, if the data did not have any bug. But above code gives an error during insertion. It is because we have some unwanted symbols which are causing problem in the insertion. It will be easier to debug the dataframe. So, we are going to follow followig steps:*
1. Load the json data in a dataframe
2. In order to demonstrate how to insert into mongoDB database and retrieve from it, we will:
>insert from the dataframe, if some error occurs, we can have a look at that cell of dataframe where the error has occured and can debug accordingly.
>We will also show how to fetch a data stored in form of collection in the mongoDB database into pandas dataframe.
3. Now we will do the analysis on dataframe we have obtained.(This dataframe can be any of the two dataframes we have, df or df2. Because both of these dataframes contain same values. I have used df because my system was taking more than 2-3 hours in storing to and retrieving from the database(intel i3, 4 GB ram)

# Step 3: Getting the data ready for analysis
## Loading data into df dataframe

In [17]:
############################################
import pandas as pd
df=pd.read_json("companies.json", lines=True)

*While storing the data into MongoDB database, we saw that the first column of dataframe has special symbol dollar which is causing a problem. Similarly all the values in _id column and few of those in 'created_at' have dollar symbol. These need to be removed.*

In [35]:
#Renamig the first column
df.columns=['ID', 'acquisition', 'acquisitions', 'alias_list', 'blog_feed_url',
       'blog_url', 'category_code', 'competitions', 'created_at',
       'crunchbase_url', 'deadpooled_day', 'deadpooled_month',
       'deadpooled_url', 'deadpooled_year', 'description', 'email_address',
       'external_links', 'founded_day', 'founded_month', 'founded_year',
       'funding_rounds', 'homepage_url', 'image', 'investments', 'ipo',
       'milestones', 'name', 'number_of_employees', 'offices', 'overview',
       'partners', 'permalink', 'phone_number', 'products', 'providerships',
       'relationships', 'screenshots', 'tag_list', 'total_money_raised',
       'twitter_username', 'updated_at', 'video_embeds']

In 'created_at' column, data type is not homogenious. Moreover, we don't require this column for processing any of our queries. So **let's drop this column**

In [86]:
df.drop(['created_at'], axis=1,inplace=True)


In [107]:
df.shape

(18801, 41)

## Pre-processing the ID column

In [76]:
fifth=[]
for i in df['ID']:
    for v in i.values():
        fifth.append(v)

In [80]:
for i in range(18801):
    df['ID'].iloc[i]=fifth[i]


/home/gyan/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [81]:
#Now, let's check whether we have data in proper format or not in ID column(atfer updation)
df['ID'].iloc[500]

'52cdef7c4bab8bd675297f7e'

Now, we have string data in ID column. Moreover created_at has been removed. Let's insert the dataframe into the database after converting it to json

In [103]:
records = json.loads(df.iloc.T.to_json()).values()
mydb.mycol.insert(records)

/home/gyan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


[ObjectId('5d00cb265b8e3d4b083da2bd'),
 ObjectId('5d00cb265b8e3d4b083da2be'),
 ObjectId('5d00cb265b8e3d4b083da2bf'),
 ObjectId('5d00cb265b8e3d4b083da2c0'),
 ObjectId('5d00cb265b8e3d4b083da2c1')]

previously we checked the list of databases and we couldn't find our database, because it had no data. let's check it once again. Now it should show the database mydatabase.

In [106]:
dblist = myclient.list_database_names()
if "mydatabase" in dblist:
    print("The database exists.")

The database exists.


## Defining function to read json into a dataframe

In [94]:
def read_mongo(db, collection, query={}, host='localhost', port=27017, username=None, password=None, no_id=True):
    """ Read from Mongo and Store into DataFrame """

    # Connect to MongoDB
    db = mydb

    # Make a query to the specific DB and Collection
    cursor = db[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']

    return df

## Reading data from dataframe

In [95]:

df2=read_mongo(mydb, 'mycol')

In [96]:
df2.head(2)

,ID,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,crunchbase_url,deadpooled_day,...,phone_number,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds
0,52cdef7c4bab8bd675297d8a,"{'price_amount': 30000000, 'price_currency_cod...",[],,http://digitalquarters.net/feed/,http://digitalquarters.net/,web,"[{'competitor': {'name': 'Wikia', 'permalink':...",http://www.crunchbase.com/company/wetpaint,NaN,...,206.859.6300,"[{'name': 'Wikison Wetpaint', 'permalink': 'we...",[],"[{'is_past': False, 'title': 'Co-Founder and V...","[{'available_sizes': [[[150, 86], 'assets/imag...","wiki, seattle, elowitz, media-industry, media-...",$39.8M,BachelrWetpaint,1386486944000,[]
1,52cdef7c4bab8bd675297d8b,None,[],Zoho ManageEngine,,,enterprise,[],http://www.crunchbase.com/company/adventnet,NaN,...,925-924-9500,[],"[{'title': 'DHFH', 'is_past': True, 'provider'...","[{'is_past': True, 'title': 'CEO and Co-Founde...","[{'available_sizes': [[[150, 94], 'assets/imag...",,$0,manageengine,1351707969000,[]


# Step 4: Analysis of Data

## 1. calculating the number of companies

In [18]:
len(list(df['name'].unique()))

17893

## 2. Calculating number of files updated between 2010 and 2013

In [19]:
from datetime import datetime

In [21]:
c=[]
for i in df['updated_at']:
    c.append(i.strftime("%Y "))

In [22]:
count=0
for i in c:
    if int(i)< 2013 and int(i)>2010:
        count+=1
print(count)

2869


## 3. Finding out the most common relationship and it's frequency

In [23]:
rel=[]
for i in df['relationships']:
    for j in i:
        rel.append(j['title'])

In [24]:
g=[]
r=set(rel)
for i in r:
    g.append(rel.count(i))

In [28]:
g.index(max(g))

13171

In [30]:
# frequency
g[13171]

3428

In [31]:
# relationship
list(r)[13171]

'CEO'

### Let's check phone_number column, to find out if we can use it to find country_code

In [38]:

df['phone_number']

0              206.859.6300
1              925-924-9500
2            1-888-204-3539
3            (415) 436-9638
4                          
5              660-675-5052
6              888.584.3150
7                          
8                      None
9                          
10                         
11                         
12                         
13                         
14                         
15           (650) 353 3206
16             310-445-7000
17                         
18                         
19                         
20                         
21           (212) 258-6000
22                         
23             650-254-5400
24           (408) 526-4000
25           (408) 349-3300
26             415-848-7000
27             415-896-3000
28         +44 207 965 2000
29             703-677-3999
                ...        
18771        1-949-498-5630
18772          650-474-0500
18773         972-9-8654904
18774         1408-980-4600
18775          510-5

**It seems, phone number can't be used for the purpose of finding country_code.**
## 4. Finding most common category_code and frequency

In [40]:
rel2=[]
for i in df['category_code']:
        rel2.append(i)

In [41]:
g2=[]
r2=set(rel2)
for i in r2:
    g2.append(rel2.count(i))

In [43]:
g2.index(max(g2))

12

In [44]:
#frequency
g2[12]

3787

In [45]:
#name of category
list(r2)[12]

'web'

## 5. Number of companies who have raised IPO

In [64]:
third=[]
for i in df['ipo']:
    if i:
        third.append(i)

In [65]:
len(third)

393

## 6. Name of Last company in the record

In [73]:
df.tail(1)['name']

18800    EnteGreat
Name: name, dtype: object

## 7. Average and median of funding_rounds

In [74]:
df['funding_rounds']

0        [{'id': 888, 'round_code': 'a', 'source_url': ...
1                                                       []
2                                                       []
3        [{'id': 1, 'round_code': 'b', 'source_url': 'h...
4        [{'id': 2, 'round_code': 'angel', 'source_url'...
5        [{'id': 225, 'round_code': 'angel', 'source_ur...
6                                                       []
7        [{'id': 6, 'round_code': 'a', 'source_url': ''...
8                                                       []
9                                                       []
10       [{'id': 261, 'round_code': 'a', 'source_url': ...
11       [{'id': 8, 'round_code': 'seed', 'source_url':...
12       [{'id': 9, 'round_code': 'a', 'source_url': 'h...
13       [{'id': 10, 'round_code': 'seed', 'source_url'...
14       [{'id': 12, 'round_code': 'b', 'source_url': '...
15       [{'id': 16, 'round_code': 'b', 'source_url': '...
16                                                      

In [99]:
fourth=[]
for i in df['funding_rounds']:
    for j in i:
        fourth.append(j['id'])

In [100]:
import statistics
#Median
statistics.median(fourth)

8079

In [101]:
#Average
statistics.mean(fourth)

15637.328522849797

<h2><center>Thanks for accompanying.</center><h2>